In [1]:
import numpy as np
import autograd.numpy as np_
from autograd import grad
import pandas as pd
import joblib
import scipy.stats as stats
from sklearn.tree import DecisionTreeClassifier


In [86]:
# lendo o CSV
df = pd.read_csv("avc-data.csv")
# One Hot Encoding dos dados (tranformando variáveis em booleanos)
df = pd.get_dummies(df)

# retirada de variáveis não booleanas do DataFrame (variáveis que não possíveis de serem representadas no OneHot Encoding)
df = df.drop(columns=['id','age','bmi','avg_glucose_level'])
df = df.dropna()
df

,hypertension,heart_disease,stroke,gender_Female,gender_Male,gender_Other,ever_married_No,ever_married_Yes,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,0,1,1,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0
1,0,0,1,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0
2,0,1,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0
3,0,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1
4,1,0,1,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5105,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0
5106,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0
5107,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0
5108,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0


In [87]:
# Criação da Matriz de Features - Todos as Variáveis, menos o resultado de se a pessoa teve um AVC ou não
df_values = df.drop(columns=['stroke'])
x = df_values.to_numpy()
x

array([[0, 1, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 1, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]], dtype=int64)

In [88]:
# Criação da Matriz de Resultados Medidos - Apenas as informações de se a pessoa teve um AVC ou não
df_stroke = df['stroke']
df_stroke = df_stroke.replace(0,-1)
y = df_stroke.to_numpy()
y

array([ 1,  1,  1, ..., -1, -1, -1], dtype=int64)

In [89]:
from sklearn.model_selection import train_test_split
# Criação de Matrizes de Treinamento e de Teste, para podermos conferir a acurácia do processo de predição
X_train, X_test, Y_train, Y_test = train_test_split(x, y, train_size=0.5)

def accuracy(y_test, y_est):
    return np.mean(np.sign(y_test)==np.sign(y_est))

In [96]:
import numpy as np
import autograd.numpy as np_
from autograd import grad

# W - Matriz de Pesos
w = np.random.randn(x.shape[1],1)

# Viés (Bias) da equação
b = 0.1

# Matriz de Features (valores medidos)
X = X_train.T*1.0

# Matriz de Resultados Medidos
Y = Y_train.T*1.0

# Cálculo do Erro - Uma equação de Reta (ax + b = y), mas com matrizes que possuem mais do que dois valores cardeais
def erro(param):
    w,b,x,y= param
    y_ = w.T@x + b
    return np_.mean((y_-y)**2)

# processo de realização da função de gradiente
g = grad(erro)

# "Learning Rate" do nosso modelo
alpha = 0.01

# Iteração de for para o aprendizado do modelo
for _ in range(10000):
    g_= g((w,b,X,Y))
    w -= alpha*g_[0]
    b -= alpha*g_[1]

# Estimativa de Y (resultado da aproximação linear)
y_est =  w.T@X_test.T + b

# checando a acurácia do modelo
accuracy(y_est,Y_test.T)

0.9542074363992172